In [146]:
# This is a project for the computational physics course Fall 2018 at the Perimeter Institute
# The goal of this project is to get familiar with Julia language features and practice them

using Plots
using DataFrames
using Dates
using StatsBase
using Statistics
##############################################################################################################################################

CITY_DATA = Dict("Chicago" => "chicago.csv", "New York City" => "new_york_city.csv", "Washington" => "washington.csv")

function load_data(city::String, skipmissing = true)
	# load data file into a dataframe
	# By default, delete the rows with missing values
	df = readtable(CITY_DATA[city])
	if skipmissing
		dropmissing!(df)
	end
	# convert the Start Time column to datetime
	dateformat = "yyyy-mm-dd HH:MM:SS"
	df[:Start_Time] = map(x->DateTime(x,dateformat), df[:Start_Time])
	df[:End_Time] = map(x->DateTime(x,dateformat), df[:End_Time])
	df
end

function time_stats(df::DataFrame)
	# Displays statistics on the most frequent times of travel
	println("\nCalculating The Most Frequent Times of Travel...\n")
	@time begin
		# display the most common month
		df[:month] = map(x->Dates.month(x), df[:Start_Time])
		popular_month = mode(df[:month])
		println("The most popular month is:$popular_month")
		# display the most common day of week
		df[:day] = map(x->Dates.dayofweek(x), df[:Start_Time])
		popular_day = mode(df[:day])
		println("The most popular day of week is:$popular_day")
	end
end
	

function station_stats(df::DataFrame)
	# Displays statistics on the most popular stations and trip
	println("\nCalculating The Most Popular Stations and Trip...\n")
	@time begin
	    #display most commonly used start station
	    popular_start_station = mode(df[:Start_Station])
	    println("The most popular start station is:$popular_start_station")
	    # display most commonly used end station
	    popular_end_station = mode(df[:End_Station])
	    println("The most popular end station is:$popular_end_station")
	    # display most frequent combination of start station and end station trip
	    trip = "From" .* df[:Start_Station] .* "to" .* df[:End_Station]
	    popular_trip = mode(trip)
	    println("The most popular combination of start and end stations is:$popular_trip")
	end
end	

function trip_duration_stats(df::DataFrame)
	# Displays statistics on the total and average trip duration
	println("\nCalculating Trip Duration...\n")
	@time begin
	N = nrow(df)
	# display total travel time
	total_travel_time = sum(df[:Trip_Duration]) / 60
	println("The total travel time (in minutes) is:$total_travel_time")
	# display mean travel time
	mean_travel_time = total_travel_time / N
	println("The mean travel time (in minutes) is:$mean_travel_time")
	end
end

function user_stats(df::DataFrame)
	# Displays statistics on bikeshare users
	println("\nCalculating User Stats...\n")  
	@time begin
		# display counts of user types
		user_types = countmap(dropmissing!(df)[:User_Type])
		@show user_types
		# display counts of gender
		if :Gender in names(df)
			gender = countmap(dropmissing!(df)[:Gender])
			@show gender
		else
			println("There is no data for gender")
		end
		# display earliest, most recent, and most common year of birth
		if :Birth_Year in names(df)
			earliest = minimum(df[:Birth_Year])
			most_recent = maximum(df[:Birth_Year])
			popular = mode(df[:Birth_Year])
			println("The earliest year of birth is $earliest \nThe most recent year of birth is $most_recent \n The most common year of birth is $popular")
		else
			println("There is no data for birth year")
		end
	end
end

function show_results(df::DataFrame)
	time_stats(df)
	station_stats(df)
	trip_duration_stats(df)
	user_stats(df)
end

# Practice some Plots
function practice_plots(df::DataFrame)
	plot(2017 .- df[df.Gender .== "Female", :][:Birth_Year],
	df[df.Gender .== "Female", :][:Trip_Duration] ./60,
	seriestype=:scatter,
	color = "red",
  	label = "Female",
  	title = "Trip Duration vs Age",
  	xlabel = "Age",
 	ylabel = "Trip Duration(min)",
  	xlims = (0,120),
  	ylims = (0,1500),
 	legend =:topright)

	plot!(2017 .- df[df.Gender .== "Male", :][:Birth_Year],
	df[df.Gender .== "Male", :][:Trip_Duration] ./60,
	seriestype=:scatter,
	color = "blue",
  	label = "Male",
 	title = "Trip Duration vs Age",
  	xlabel = "Age",
  	ylabel = "Trip Duration(min)",
 	xlims = (0,120),
	ylims = (0,1500),
	legend =:topright)
end
##############################################################################################################################################

practice_plots (generic function with 1 method)

In [30]:
df1= load_data("Chicago")

┌ Warning: readtable is deprecated, use CSV.read from the CSV package instead
│   caller = load_data(::String, ::Bool) at In[29]:15
└ @ Main .\In[29]:15


,x,Start_Time,End_Time,Trip_Duration,Start_Station,End_Station,User_Type,Gender,Birth_Year
,Int64⍰,DateTime,DateTime,Int64⍰,String⍰,String⍰,String⍰,String⍰,Float64⍰
1,1423854,2017-06-23T15:09:32,2017-06-23T15:14:53,321,Wood St & Hubbard St,Damen Ave & Chicago Ave,Subscriber,Male,1992.0
2,955915,2017-05-25T18:19:03,2017-05-25T18:45:53,1610,Theater on the Lake,Sheffield Ave & Waveland Ave,Subscriber,Female,1992.0
3,9031,2017-01-04T08:27:49,2017-01-04T08:34:45,416,May St & Taylor St,Wood St & Taylor St,Subscriber,Male,1981.0
4,304487,2017-03-06T13:49:38,2017-03-06T13:55:28,350,Christiana Ave & Lawrence Ave,St. Louis Ave & Balmoral Ave,Subscriber,Male,1986.0
5,45207,2017-01-17T14:53:07,2017-01-17T15:02:01,534,Clark St & Randolph St,Desplaines St & Jackson Blvd,Subscriber,Male,1975.0
6,1473887,2017-06-26T09:01:20,2017-06-26T09:11:06,586,Clinton St & Washington Blvd,Canal St & Taylor St,Subscriber,Male,1990.0
7,961916,2017-05-26T09:41:44,2017-05-26T09:46:25,281,Ashland Ave & Lake St,Wood St & Hubbard St,Subscriber,Female,1983.0
8,606841,2017-04-20T16:08:51,2017-04-20T16:20:20,689,Sedgwick St & Huron St,Halsted St & Blackhawk St (*),Subscriber,Male,1984.0
9,135470,2017-02-06T18:00:47,2017-02-06T18:09:00,493,Stetson Ave & South Water St,Clinton St & Washington Blvd,Subscriber,Male,1979.0


In [31]:
df2 = load_data("Chicago", true)

,x,Start_Time,End_Time,Trip_Duration,Start_Station,End_Station,User_Type,Gender,Birth_Year
,Int64⍰,DateTime,DateTime,Int64⍰,String⍰,String⍰,String⍰,String⍰,Float64⍰
1,1423854,2017-06-23T15:09:32,2017-06-23T15:14:53,321,Wood St & Hubbard St,Damen Ave & Chicago Ave,Subscriber,Male,1992.0
2,955915,2017-05-25T18:19:03,2017-05-25T18:45:53,1610,Theater on the Lake,Sheffield Ave & Waveland Ave,Subscriber,Female,1992.0
3,9031,2017-01-04T08:27:49,2017-01-04T08:34:45,416,May St & Taylor St,Wood St & Taylor St,Subscriber,Male,1981.0
4,304487,2017-03-06T13:49:38,2017-03-06T13:55:28,350,Christiana Ave & Lawrence Ave,St. Louis Ave & Balmoral Ave,Subscriber,Male,1986.0
5,45207,2017-01-17T14:53:07,2017-01-17T15:02:01,534,Clark St & Randolph St,Desplaines St & Jackson Blvd,Subscriber,Male,1975.0
6,1473887,2017-06-26T09:01:20,2017-06-26T09:11:06,586,Clinton St & Washington Blvd,Canal St & Taylor St,Subscriber,Male,1990.0
7,961916,2017-05-26T09:41:44,2017-05-26T09:46:25,281,Ashland Ave & Lake St,Wood St & Hubbard St,Subscriber,Female,1983.0
8,606841,2017-04-20T16:08:51,2017-04-20T16:20:20,689,Sedgwick St & Huron St,Halsted St & Blackhawk St (*),Subscriber,Male,1984.0
9,135470,2017-02-06T18:00:47,2017-02-06T18:09:00,493,Stetson Ave & South Water St,Clinton St & Washington Blvd,Subscriber,Male,1979.0


In [32]:
head(df1)

,x,Start_Time,End_Time,Trip_Duration,Start_Station,End_Station,User_Type,Gender,Birth_Year
,Int64⍰,DateTime,DateTime,Int64⍰,String⍰,String⍰,String⍰,String⍰,Float64⍰
1,1423854,2017-06-23T15:09:32,2017-06-23T15:14:53,321,Wood St & Hubbard St,Damen Ave & Chicago Ave,Subscriber,Male,1992.0
2,955915,2017-05-25T18:19:03,2017-05-25T18:45:53,1610,Theater on the Lake,Sheffield Ave & Waveland Ave,Subscriber,Female,1992.0
3,9031,2017-01-04T08:27:49,2017-01-04T08:34:45,416,May St & Taylor St,Wood St & Taylor St,Subscriber,Male,1981.0
4,304487,2017-03-06T13:49:38,2017-03-06T13:55:28,350,Christiana Ave & Lawrence Ave,St. Louis Ave & Balmoral Ave,Subscriber,Male,1986.0
5,45207,2017-01-17T14:53:07,2017-01-17T15:02:01,534,Clark St & Randolph St,Desplaines St & Jackson Blvd,Subscriber,Male,1975.0
6,1473887,2017-06-26T09:01:20,2017-06-26T09:11:06,586,Clinton St & Washington Blvd,Canal St & Taylor St,Subscriber,Male,1990.0


In [33]:
tail(df1)

,x,Start_Time,End_Time,Trip_Duration,Start_Station,End_Station,User_Type,Gender,Birth_Year
,Int64⍰,DateTime,DateTime,Int64⍰,String⍰,String⍰,String⍰,String⍰,Float64⍰
1,723217,2017-05-03T19:15:23,2017-05-03T19:29:32,849,Milwaukee Ave & Wabansia Ave,Ashland Ave & Chicago Ave,Subscriber,Female,1982.0
2,1327248,2017-06-17T16:44:16,2017-06-17T16:50:57,401,Sheffield Ave & Willow St,Halsted St & Dickens Ave,Subscriber,Male,1955.0
3,935129,2017-05-23T17:08:04,2017-05-23T17:11:37,213,Indiana Ave & 31st St,MLK Jr Dr & 29th St,Subscriber,Male,1988.0
4,64825,2017-01-21T13:18:00,2017-01-21T13:27:50,590,Orleans St & Elm St (*),Sheffield Ave & Webster Ave,Subscriber,Male,1965.0
5,159685,2017-02-12T09:59:01,2017-02-12T10:21:49,1368,Ravenswood Ave & Lawrence Ave,Stockton Dr & Wrightwood Ave,Subscriber,Female,1988.0
6,1021247,2017-05-30T12:38:28,2017-05-30T12:55:36,1028,Wacker Dr & Washington St,Damen Ave & Division St,Subscriber,Male,1984.0


In [34]:
describe(df1)

,variable,mean,min,median,max,nunique,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Union…,Union…,DataType
1,x,7.42185e5,4,7.2265e5,1551500,,0,Int64
2,Start_Time,,2017-01-01T00:07:57,,2017-06-30T23:59:01,234366,,DateTime
3,End_Time,,2017-01-01T00:20:53,,2017-07-01T00:24:22,233978,,DateTime
4,Trip_Duration,699.391,60,565.0,85742,,0,Int64
5,Start_Station,,2112 W Peterson Ave,,Yates Blvd & 75th St,560,0,String
6,End_Station,,2112 W Peterson Ave,,Yates Blvd & 75th St,562,0,String
7,User_Type,,Customer,,Subscriber,3,0,String
8,Gender,,Female,,Male,2,0,String
9,Birth_Year,1980.86,1899.0,1984.0,2016.0,,0,Float64


In [108]:
df1[(df1.Birth_Year .> 1990) .& (df1.Trip_Duration .> 1000) .& (df1.Gender .== "Female"), :][1:2,:]

,x,Start_Time,End_Time,Trip_Duration,Start_Station,End_Station,User_Type,Gender,Birth_Year
,Int64⍰,DateTime,DateTime,Int64⍰,String⍰,String⍰,String⍰,String⍰,Float64⍰
1,955915,2017-05-25T18:19:03,2017-05-25T18:45:53,1610,Michigan Ave & Oak St,Sheffield Ave & Waveland Ave,Subscriber,Female,1992.0
2,849468,2017-05-16T06:36:17,2017-05-16T06:59:34,1397,Michigan Ave & Oak St,Halsted St & Wrightwood Ave,Subscriber,Female,1992.0


In [132]:
col_1 = df1[(df1.Start_Station .== "Michigan Ave & Oak St") .& (df1.End_Station .== "Halsted St & Wrightwood Ave") ,:][1:2,:]

,x,Start_Time,End_Time,Trip_Duration,Start_Station,End_Station,User_Type,Gender,Birth_Year
,Int64⍰,DateTime,DateTime,Int64⍰,String⍰,String⍰,String⍰,String⍰,Float64⍰
1,849468,2017-05-16T06:36:17,2017-05-16T06:59:34,1397,Michigan Ave & Oak St,Halsted St & Wrightwood Ave,Subscriber,Female,1992.0
2,261757,2017-02-25T19:01:19,2017-02-25T19:08:10,411,Michigan Ave & Oak St,Halsted St & Wrightwood Ave,Subscriber,Male,1970.0


In [133]:
col_2 = df1[(df1.Start_Station .== "Michigan Ave & Oak St") .& (df1.End_Station .== "Lincoln Ave & Addison St") ,:][1:2,:]

,x,Start_Time,End_Time,Trip_Duration,Start_Station,End_Station,User_Type,Gender,Birth_Year
,Int64⍰,DateTime,DateTime,Int64⍰,String⍰,String⍰,String⍰,String⍰,Float64⍰
1,1377062,2017-06-20T18:29:56,2017-06-20T19:00:22,1826,Michigan Ave & Oak St,Lincoln Ave & Addison St,Subscriber,Female,1991.0
2,1396870,2017-06-21T19:39:08,2017-06-21T20:11:52,1964,Michigan Ave & Oak St,Lincoln Ave & Addison St,Subscriber,Female,1996.0


In [135]:
join(col_1, col_2, on = :Start_Station, kind = :inner)

,x,Start_Time,End_Time,Trip_Duration,Start_Station,End_Station,User_Type,Gender,Birth_Year,x_1,Start_Time_1,End_Time_1,Trip_Duration_1,End_Station_1,User_Type_1,Gender_1,Birth_Year_1
,Int64⍰,DateTime,DateTime,Int64⍰,String⍰,String⍰,String⍰,String⍰,Float64⍰,Int64⍰,DateTime,DateTime,Int64⍰,String⍰,String⍰,String⍰,Float64⍰
1,849468,2017-05-16T06:36:17,2017-05-16T06:59:34,1397,Michigan Ave & Oak St,Halsted St & Wrightwood Ave,Subscriber,Female,1992.0,1377062,2017-06-20T18:29:56,2017-06-20T19:00:22,1826,Lincoln Ave & Addison St,Subscriber,Female,1991.0
2,849468,2017-05-16T06:36:17,2017-05-16T06:59:34,1397,Michigan Ave & Oak St,Halsted St & Wrightwood Ave,Subscriber,Female,1992.0,1396870,2017-06-21T19:39:08,2017-06-21T20:11:52,1964,Lincoln Ave & Addison St,Subscriber,Female,1996.0
3,261757,2017-02-25T19:01:19,2017-02-25T19:08:10,411,Michigan Ave & Oak St,Halsted St & Wrightwood Ave,Subscriber,Male,1970.0,1377062,2017-06-20T18:29:56,2017-06-20T19:00:22,1826,Lincoln Ave & Addison St,Subscriber,Female,1991.0
4,261757,2017-02-25T19:01:19,2017-02-25T19:08:10,411,Michigan Ave & Oak St,Halsted St & Wrightwood Ave,Subscriber,Male,1970.0,1396870,2017-06-21T19:39:08,2017-06-21T20:11:52,1964,Lincoln Ave & Addison St,Subscriber,Female,1996.0


In [143]:
join(col_1, col_2, on = :End_Station, kind = :outer)

,x,Start_Time,End_Time,Trip_Duration,Start_Station,End_Station,User_Type,Gender,Birth_Year,x_1,Start_Time_1,End_Time_1,Trip_Duration_1,Start_Station_1,User_Type_1,Gender_1,Birth_Year_1
,Int64⍰,DateTime⍰,DateTime⍰,Int64⍰,String⍰,String⍰,String⍰,String⍰,Float64⍰,Int64⍰,DateTime⍰,DateTime⍰,Int64⍰,String⍰,String⍰,String⍰,Float64⍰
1,849468,2017-05-16T06:36:17,2017-05-16T06:59:34,1397,Michigan Ave & Oak St,Halsted St & Wrightwood Ave,Subscriber,Female,1992.0,missing,missing,missing,missing,missing,missing,missing,missing
2,261757,2017-02-25T19:01:19,2017-02-25T19:08:10,411,Michigan Ave & Oak St,Halsted St & Wrightwood Ave,Subscriber,Male,1970.0,missing,missing,missing,missing,missing,missing,missing,missing
3,missing,missing,missing,missing,missing,Lincoln Ave & Addison St,missing,missing,missing,1377062,2017-06-20T18:29:56,2017-06-20T19:00:22,1826,Michigan Ave & Oak St,Subscriber,Female,1991.0
4,missing,missing,missing,missing,missing,Lincoln Ave & Addison St,missing,missing,missing,1396870,2017-06-21T19:39:08,2017-06-21T20:11:52,1964,Michigan Ave & Oak St,Subscriber,Female,1996.0


In [144]:
join(col_1, col_2, on = :Gender, kind = :anti)

,x,Start_Time,End_Time,Trip_Duration,Start_Station,End_Station,User_Type,Gender,Birth_Year
,Int64⍰,DateTime,DateTime,Int64⍰,String⍰,String⍰,String⍰,String⍰,Float64⍰
1,261757,2017-02-25T19:01:19,2017-02-25T19:08:10,411,Michigan Ave & Oak St,Halsted St & Wrightwood Ave,Subscriber,Male,1970.0


In [141]:
join(col_1, col_2, kind = :cross, makeunique=true)

,x,Start_Time,End_Time,Trip_Duration,Start_Station,End_Station,User_Type,Gender,Birth_Year,x_1,Start_Time_1,End_Time_1,Trip_Duration_1,Start_Station_1,End_Station_1,User_Type_1,Gender_1,Birth_Year_1
,Int64⍰,DateTime,DateTime,Int64⍰,String⍰,String⍰,String⍰,String⍰,Float64⍰,Int64⍰,DateTime,DateTime,Int64⍰,String⍰,String⍰,String⍰,String⍰,Float64⍰
1,849468,2017-05-16T06:36:17,2017-05-16T06:59:34,1397,Michigan Ave & Oak St,Halsted St & Wrightwood Ave,Subscriber,Female,1992.0,1377062,2017-06-20T18:29:56,2017-06-20T19:00:22,1826,Michigan Ave & Oak St,Lincoln Ave & Addison St,Subscriber,Female,1991.0
2,849468,2017-05-16T06:36:17,2017-05-16T06:59:34,1397,Michigan Ave & Oak St,Halsted St & Wrightwood Ave,Subscriber,Female,1992.0,1396870,2017-06-21T19:39:08,2017-06-21T20:11:52,1964,Michigan Ave & Oak St,Lincoln Ave & Addison St,Subscriber,Female,1996.0
3,261757,2017-02-25T19:01:19,2017-02-25T19:08:10,411,Michigan Ave & Oak St,Halsted St & Wrightwood Ave,Subscriber,Male,1970.0,1377062,2017-06-20T18:29:56,2017-06-20T19:00:22,1826,Michigan Ave & Oak St,Lincoln Ave & Addison St,Subscriber,Female,1991.0
4,261757,2017-02-25T19:01:19,2017-02-25T19:08:10,411,Michigan Ave & Oak St,Halsted St & Wrightwood Ave,Subscriber,Male,1970.0,1396870,2017-06-21T19:39:08,2017-06-21T20:11:52,1964,Michigan Ave & Oak St,Lincoln Ave & Addison St,Subscriber,Female,1996.0


In [148]:
by(df1, :Gender, df1 -> mean(df1.Trip_Duration))

,Gender,x1
,String⍰,Float64
1,Male,673.212
2,Female,781.516


In [150]:
by(df1, :User_Type) do df1
    DataFrame(m = mean(df1.Trip_Duration), s² = var(df1.Trip_Duration))
end

,User_Type,m,s²
,String⍰,Float64,Float64
1,Subscriber,699.351,9.83955e5
2,Customer,807.637,3.76558e5
3,Dependent,311.0,NaN


In [159]:
aggregate(col_1, :Birth_Year, typeof)

,Birth_Year,x_typeof,Start_Time_typeof,End_Time_typeof,Trip_Duration_typeof,Start_Station_typeof,End_Station_typeof,User_Type_typeof,Gender_typeof
,Float64⍰,DataType,DataType,DataType,DataType,DataType,DataType,DataType,DataType
1,1992.0,"Array{Union{Missing, Int64},1}","Array{DateTime,1}","Array{DateTime,1}","Array{Union{Missing, Int64},1}","Array{Union{Missing, String},1}","Array{Union{Missing, String},1}","Array{Union{Missing, String},1}","Array{Union{Missing, String},1}"
2,1970.0,"Array{Union{Missing, Int64},1}","Array{DateTime,1}","Array{DateTime,1}","Array{Union{Missing, Int64},1}","Array{Union{Missing, String},1}","Array{Union{Missing, String},1}","Array{Union{Missing, String},1}","Array{Union{Missing, String},1}"


In [160]:
typeof(missing)

Missing

In [164]:
sort!(df1, (:Trip_Duration, :Start_Station, :Birth_Year))

,x,Start_Time,End_Time,Trip_Duration,Start_Station,End_Station,User_Type,Gender,Birth_Year
,Int64⍰,DateTime,DateTime,Int64⍰,String⍰,String⍰,String⍰,String⍰,Float64⍰
1,104080,2017-01-30T11:58:04,2017-01-30T11:59:04,60,Michigan Ave & Oak St,Ashland Ave & 21st St,Subscriber,Male,1949.0
2,1399109,2017-06-21T22:24:54,2017-06-21T22:25:54,60,Michigan Ave & Oak St,Clinton St & Madison St,Subscriber,Male,1955.0
3,1286867,2017-06-15T09:17:38,2017-06-15T09:18:38,60,Michigan Ave & Oak St,Western Ave & Winnebago Ave,Subscriber,Male,1956.0
4,124303,2017-02-03T16:53:51,2017-02-03T16:54:51,60,Michigan Ave & Oak St,Canal St & Adams St,Subscriber,Male,1964.0
5,599295,2017-04-19T18:27:12,2017-04-19T18:28:12,60,Michigan Ave & Oak St,Orleans St & Merchandise Mart Plaza,Subscriber,Male,1972.0
6,1466472,2017-06-25T17:46:40,2017-06-25T17:47:40,60,Michigan Ave & Oak St,State St & 19th St,Subscriber,Female,1972.0
7,1261790,2017-06-13T13:15:20,2017-06-13T13:16:20,60,Michigan Ave & Oak St,Larrabee St & Webster Ave,Subscriber,Female,1973.0
8,971879,2017-05-27T12:24:34,2017-05-27T12:25:34,60,Michigan Ave & Oak St,Theater on the Lake,Subscriber,Male,1977.0
9,143331,2017-02-08T11:33:55,2017-02-08T11:34:55,60,Michigan Ave & Oak St,Peoria St & Jackson Blvd,Subscriber,Male,1978.0


In [165]:
sort!(df1, (:Trip_Duration, :Start_Station, :Birth_Year), rev = (true, false, false))

,x,Start_Time,End_Time,Trip_Duration,Start_Station,End_Station,User_Type,Gender,Birth_Year
,Int64⍰,DateTime,DateTime,Int64⍰,String⍰,String⍰,String⍰,String⍰,Float64⍰
1,217922,2017-02-19T16:02:00,2017-02-20T15:51:02,85742,Michigan Ave & Oak St,Halsted St & Blackhawk St (*),Subscriber,Female,1972.0
2,512635,2017-04-11T18:45:19,2017-04-12T18:31:31,85572,Michigan Ave & Oak St,Lake Shore Dr & Wellington Ave,Subscriber,Male,1984.0
3,10473,2017-01-04T17:06:17,2017-01-05T15:32:34,80777,Michigan Ave & Oak St,Clark St & Schiller St,Subscriber,Male,1976.0
4,49262,2017-01-18T09:14:54,2017-01-19T07:25:07,79813,Michigan Ave & Oak St,Clark St & Lake St,Subscriber,Female,1971.0
5,264867,2017-02-26T18:40:42,2017-02-27T16:18:47,77885,Michigan Ave & Oak St,Clark St & Grace St,Subscriber,Male,1992.0
6,22147,2017-01-09T17:00:34,2017-01-10T13:39:56,74362,Michigan Ave & Oak St,Larrabee St & North Ave,Subscriber,Male,1970.0
7,135434,2017-02-06T17:57:14,2017-02-07T14:22:50,73536,Michigan Ave & Oak St,Western Ave & Winnebago Ave,Subscriber,Female,1983.0
8,375392,2017-03-21T16:27:03,2017-03-22T12:37:03,72600,Michigan Ave & Oak St,Broadway & Waveland Ave,Subscriber,Male,1963.0
9,1362988,2017-06-19T21:48:45,2017-06-20T17:32:27,71022,Michigan Ave & Oak St,Bissell St & Armitage Ave,Subscriber,Male,1969.0


In [169]:
d = stack(df1, 5:6)

,variable,value,x,Start_Time,End_Time,Trip_Duration,User_Type,Gender,Birth_Year
,Symbol,String⍰,Int64⍰,DateTime,DateTime,Int64⍰,String⍰,String⍰,Float64⍰
1,Start_Station,Michigan Ave & Oak St,217922,2017-02-19T16:02:00,2017-02-20T15:51:02,85742,Subscriber,Female,1972.0
2,Start_Station,Michigan Ave & Oak St,512635,2017-04-11T18:45:19,2017-04-12T18:31:31,85572,Subscriber,Male,1984.0
3,Start_Station,Michigan Ave & Oak St,10473,2017-01-04T17:06:17,2017-01-05T15:32:34,80777,Subscriber,Male,1976.0
4,Start_Station,Michigan Ave & Oak St,49262,2017-01-18T09:14:54,2017-01-19T07:25:07,79813,Subscriber,Female,1971.0
5,Start_Station,Michigan Ave & Oak St,264867,2017-02-26T18:40:42,2017-02-27T16:18:47,77885,Subscriber,Male,1992.0
6,Start_Station,Michigan Ave & Oak St,22147,2017-01-09T17:00:34,2017-01-10T13:39:56,74362,Subscriber,Male,1970.0
7,Start_Station,Michigan Ave & Oak St,135434,2017-02-06T17:57:14,2017-02-07T14:22:50,73536,Subscriber,Female,1983.0
8,Start_Station,Michigan Ave & Oak St,375392,2017-03-21T16:27:03,2017-03-22T12:37:03,72600,Subscriber,Male,1963.0
9,Start_Station,Michigan Ave & Oak St,1362988,2017-06-19T21:48:45,2017-06-20T17:32:27,71022,Subscriber,Male,1969.0


In [170]:
head(d)

,variable,value,x,Start_Time,End_Time,Trip_Duration,User_Type,Gender,Birth_Year
,Symbol,String⍰,Int64⍰,DateTime,DateTime,Int64⍰,String⍰,String⍰,Float64⍰
1,Start_Station,Michigan Ave & Oak St,217922,2017-02-19T16:02:00,2017-02-20T15:51:02,85742,Subscriber,Female,1972.0
2,Start_Station,Michigan Ave & Oak St,512635,2017-04-11T18:45:19,2017-04-12T18:31:31,85572,Subscriber,Male,1984.0
3,Start_Station,Michigan Ave & Oak St,10473,2017-01-04T17:06:17,2017-01-05T15:32:34,80777,Subscriber,Male,1976.0
4,Start_Station,Michigan Ave & Oak St,49262,2017-01-18T09:14:54,2017-01-19T07:25:07,79813,Subscriber,Female,1971.0
5,Start_Station,Michigan Ave & Oak St,264867,2017-02-26T18:40:42,2017-02-27T16:18:47,77885,Subscriber,Male,1992.0
6,Start_Station,Michigan Ave & Oak St,22147,2017-01-09T17:00:34,2017-01-10T13:39:56,74362,Subscriber,Male,1970.0


In [171]:
tail(d)

,variable,value,x,Start_Time,End_Time,Trip_Duration,User_Type,Gender,Birth_Year
,Symbol,String⍰,Int64⍰,DateTime,DateTime,Int64⍰,String⍰,String⍰,Float64⍰
1,End_Station,Lake Park Ave & 56th St,681705,2017-04-27T08:35:22,2017-04-27T08:36:22,60,Subscriber,Male,1991.0
2,End_Station,State St & 33rd St,704384,2017-05-01T15:23:35,2017-05-01T15:24:35,60,Subscriber,Male,1991.0
3,End_Station,Adler Planetarium,914807,2017-05-21T20:30:29,2017-05-21T20:31:29,60,Subscriber,Male,1992.0
4,End_Station,Campbell Ave & North Ave,1459207,2017-06-25T13:05:00,2017-06-25T13:06:00,60,Subscriber,Female,1992.0
5,End_Station,Streeter Dr & Grand Ave,1456787,2017-06-25T11:21:04,2017-06-25T11:22:04,60,Subscriber,Female,1993.0
6,End_Station,LaSalle St & Washington St,1273518,2017-06-14T07:33:23,2017-06-14T07:34:23,60,Subscriber,Female,1995.0


In [172]:
show_results(df1)


Calculating The Most Frequent Times of Travel...

The most popular month is:6
The most popular day of week is:2
  0.115023 seconds (255.82 k allocations: 16.741 MiB)

Calculating The Most Popular Stations and Trip...

The most popular start station is:Michigan Ave & Oak St
The most popular end station is:Clinton St & Washington Blvd
The most popular combination of start and end stations is:FromMichigan Ave & Oak SttoClinton St & Washington Blvd
  0.512554 seconds (1.41 M allocations: 76.407 MiB, 6.89% gc time)

Calculating Trip Duration...

The total travel time (in minutes) is:2.7853005833333335e6
The mean travel time (in minutes) is:11.656513481315322
  0.011856 seconds (23.41 k allocations: 1.183 MiB)

Calculating User Stats...

user_types = Dict{Union{Missing, String},Int64}("Subscriber"=>238856,"Dependent"=>1,"Customer"=>91)
gender = Dict{Union{Missing, String},Int64}("Female"=>57758,"Male"=>181190)
The earliest year of birth is 1899.0 
The most recent year of birth is 2016.0 
 T